# LLM functions and chat objects

Anton Antonov   
["Jupyter::Chatbook" Raku package at GitHub](https://github.com/antononcube/Raku-Jupyter-Chatbook)   
August, September 2023

------

## Introduction

In this notebook we show how Large Language Models (LLMs) functions and LLM chat objects can be created and used in a notebook. This notebook is a ***chatbook*** created with the Raku package ["Jupyter::Chatbook"](https://raku.land/zef:antononcube/Jupyter::Chatbook), [AAp1].

A "streamlined" way to utilize LLMs is with the package["LLM::Functions"](https://raku.land/zef:antononcube/LLM::Functions), [AA1, AAp2]. I.e. without using specific, dedicated packages for accessing LLMs like those of ["WWW::OpenAI"](https://raku.land/zef:antononcube/WWW::OpenAI), [AAp3], and ["WWW::PaLM"](https://raku.land/zef:antononcube/WWW::PaLM), [AAp4].

Chatbooks load in their initialization phase the package 
["LLM::Functions"](https://github.com/antononcube/Raku-LLM-Functions), [AAp2].
Also, in the initialization phase are loaded the packages 
["Clipboard"](https://github.com/antononcube/Raku-Clipboard), [AAp5],
["Data::Translators"](https://github.com/antononcube/Raku-Data-Translators), [AAp6],
["Data::TypeSystem"](https://github.com/antononcube/Raku-Data-Translators), [AAp7],
["Text::Plot"](https://github.com/antononcube/Raku-Text-Plot), [AAp8],
and 
["Text::SubParsers"](https://github.com/antononcube/Raku-Text-SubParsers), [AAp9], 
that can be used to post-process LLM outputs.

**Remark:** For LLM functions and chat objects the functionalities of ["Jupyter::Chatbook"](https://github.com/bduggan/raku-jupyter-kernel), [BDp1], suffice. In other words, the "chatbook" extensions provided by "Jupyter::Chatbook", [AAp1], are not needed.

**Remark:** The LLM functions and chat objects are provided by the package ["LLM::Functions"](https://raku.land/zef:antononcube/LLM::Functions), [AA1, AAp2], which in turn uses the packages ["WWW::OpenAI"](https://raku.land/zef:antononcube/WWW::OpenAI), [AAp3], and ["WWW::PaLM"](https://raku.land/zef:antononcube/WWW::PaLM), [AAp4].

**Remark:** The default API keys for the LLM functions and chat objects are taken from the Operating System (OS) environmental variables `OPENAI_API_KEY` and `PALM_API_KEY`. The api keys can also be specified using LLM evaluator and configuration options and objects; see [AA1, AAp2].


------ 

## LLM functions



Here is an example of an LLM function that takes 3 arguments:

In [1]:
my &f1 = llm-function({"What is the $^a of $^b in $^c. Give numerical answer only."})

-> **@args, *%args { #`(Block|6307365437456) ... }

Here the LLM function defined above is "concretized" with argument values:

In [4]:
&f1('GDP', 'California, USA', '2020')

The GDP of California, USA in 2020 was $3.2 trillion.

Here we call repeatedly `&f1` in order to Lake Mead levels: 

In [5]:
'level' X 'Lake Mead' X (1990...2010) ==> map({ $_.tail => &f1(|$_) }) ==> my @levels;

(1990 => In 1990, the level of Lake Mead was approximately 1,206 feet. 1991 => In 1991, the level of Lake Mead was approximately 1,204.80 feet. 1992 => The level of Lake Mead in 1992 was approximately 1,206 feet. 1993 => The water level of Lake Mead in 1993 was approximately 1,211 feet. 1994 => In 1994, the level of Lake Mead was approximately 1,205 feet. 1995 => In 1995, the level of Lake Mead was approximately 1,207 feet (368 meters). 1996 => The level of Lake Mead in 1996 was approximately 1,204 feet. 1997 => In 1997, the level of Lake Mead was approximately 1,216 feet. 1998 => In 1998, the level of Lake Mead was approximately 1,207 feet. 1999 => In 1999, the level of Lake Mead was at approximately 1,210 feet. 2000 => In 2000, the level of Lake Mead was approximately 1,214 feet. 2001 => In 2001, the level of Lake Mead was approximately 1,205 feet. 2002 => In 2002, the level of Lake Mead was approximately 1,207 feet. 2003 => In 2003, the level of Lake Mead was approximately 1,200 fee

Here we extract numbers of the obtained LLM results and plot the levels over the corresponding years:

In [6]:
@levels.map({ ($_.key, sub-parser('GeneralNumber', :drop).process($_.value).head) }) ==> text-list-plot(width=>60)

+---+------------+-----------+------------+------------+---+         
|                                                          |         
+                                                    * *   +  2010.00
|                                               * *        |         
+                                         *  *             +  2005.00
|                                    *  *                  |         
|                               *  *                       |         
+                            *                             +  2000.00
|                       *  *                               |         
|                  *  *                                    |         
+             *  *                                         +  1995.00
|        * *                                               |         
+   * *                                                    +  1990.00
|                                                          |         
+---+------------+--

Here we defined another LLM function, and since the LLM function produces (or it is suppossed to produce) HTML code we use the magic spec `%% html`:

In [8]:
%% html

my &ftbl = llm-function({"The HTML code of a random table with $^a rows and $^b columns is : "}, e => 'ChatGPT');

&ftbl(4,3)

Column 1,Column 2,Column 3
"Row 1, Cell 1","Row 1, Cell 2","Row 1, Cell 3"
"Row 2, Cell 1","Row 2, Cell 2","Row 2, Cell 3"
"Row 3, Cell 1","Row 3, Cell 2","Row 3, Cell 3"
"Row 4, Cell 1","Row 4, Cell 2","Row 4, Cell 3"


Detailed examples of LLM workflows can be found in the article ["Workflows with LLM functions"](https://rakuforprediction.wordpress.com/2023/08/01/workflows-with-llm-functions/), [AA1].

------

## Number extraction from LLM responses

Often LLM return larger number with comma delimiters between the digits. Here is an example: 

In [9]:
my $pop = llm-function()("What is the population of Niger?")

As of July 2021, the estimated population of Niger is approximately 25.5 million people.

One way to extract the numbers from those responses is to use the token `<local-number>` provided by the package ["Intl::Token::Number"](https://raku.land/zef:guifa/Intl::Token::Number), [MSp1]:

In [10]:
use Intl::Token::Number;

$pop ~~ m:g/ <local-number> /

(｢2021｣
 local-number => ｢2021｣ ｢25.5｣
 local-number => ｢25.5｣)

Alternatively, a sub-parser from the packatge ["Text::SubParsers"](https://raku.land/zef:antononcube/Text::SubParsers), [AAp8], can be used:

In [11]:
sub-parser(Whatever).subparse($pop).raku

$["As of", DateTime.new(2021,7,1,0,0,0), ", the estimated population of Niger is approximately", 25.5, "million people."]

-------

## LLM chat objects

LLM chat objects are used to provide context for LLM interactions with different messages.
LLM chat objects keep track of the conversation history, so LLM can better understand the meaning of each message.

For more involved examples of using LLM chat objects see the article ["Number guessing games: PaLM vs ChatGPT"](https://rakuforprediction.wordpress.com/2023/08/06/number-guessing-games-palm-vs-chatgpt/), [AA2].


Make a chat object:

In [12]:
my $chatObj = llm-chat(
    conf=>'ChatPaLM', 
    prompt => 'You are Raku coding instructor. You are the best Raku programmer and you know Raku documentation very well. You answers are concise, mostly with code');

LLM::Functions::Chat(chat-id = , llm-evaluator.conf.name = chatpalm, messages.elems = 0)

Here we give the chat object a message, and specify the output to be in Markdown format using the magic spec `%% markdown`:

In [13]:
%% markdown
$chatObj.eval('How do you get the characters of a string?')

To get the characters of a string in Raku, you can use the `chars` method. The `chars` method returns a list of all the characters in the string. For example, if you have a string called `"Hello, world!"`, you can use the `chars` method to get a list of all the characters in the string:

```
my @characters = "Hello, world!".chars;
```

The `@characters` variable will now contain a list of all the characters in the string, in order. You can then access individual characters in the list using the index. For example, the first character in the string is `'H'`, so you can access it using the index `0`:

```
say @characters[0]; # => H
```

The last character in the string is `'!'`, so you can access it using the index `-1`:

```
say @characters[-1]; # => !
```

You can also use the `chars` method to get the length of a string. The length of a string is the number of characters in the string. For example, the length of the string `"Hello, world!"` is 12:

```
say "Hello, world!".chars; # => 12
```

You can also use the `chars` method to iterate over the characters in a string. For example, you can use the `for` loop to iterate over the characters in the string `"Hello, world!"`:

```
for my $character ( "Hello, world!".chars ) {
  say $character;
}
```

This will print the following output:

```
H
e
l
l
o
,
 
w
o
r
l
d
!
```

You can also use the `map` function to iterate over the characters in a string. For example, you can use the `map` function to convert all the uppercase letters in the string `"Hello, world!"` to lowercase letters:

```
say "Hello, world!".chars.map { $_.downcase };
```

This will print the following output:

```
hello, world!
```

In [14]:
%% markdown
$chatObj.eval('How do you make the original word from the characters in the previous example')

To make the original word from the characters in the previous example, you can use the `join` method. The `join` method takes a list of characters and returns a string that contains all the characters in the list, joined together with the specified separator. For example, if you have a list of characters called `@characters` that contains the characters `'H'`, `'e'`, `'l'`, `'l'`, `'o'`, `','`, `' '`, `'w'`, `'o'`, `'r'`, `'l'`, `'d'`, `'!'`, you can use the `join` method to join the characters together with the space character to get the original string:

```
say " ".join @characters;
```

This will print the following output:

```
Hello, world!
```

You can also use the `join` method to join the characters together with the empty string to get a string that contains all the characters in the list, without any separators:

```
say join @characters;
```

This will print the following output:

```
HELLO, WORLD!
```

-----

## References

### Articles

[AA1] Anton Antonov,
["Workflows with LLM functions"](https://rakuforprediction.wordpress.com/2023/08/01/workflows-with-llm-functions/),
(2023),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA2] Anton Antonov,
["Number guessing games: PaLM vs ChatGPT"](https://rakuforprediction.wordpress.com/2023/08/06/number-guessing-games-palm-vs-chatgpt/),
(2023),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov,
[Jupyter::Chatbook Raku package](https://github.com/antononcube/Raku-Jupyter-Chatbook),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[WWW::PaLM Raku package](https://github.com/antononcube/Raku-WWW-PaLM),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp5] Anton Antonov,
[Clipboard Raku package](https://github.com/antononcube/Raku-Clipboard),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp6] Anton Antonov,
[Data::Translators Raku package](https://github.com/antononcube/Raku-Data-Translators),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp7] Anton Antonov,
[Data::TypeSystem Raku package](https://github.com/antononcube/Raku-Data-TypeSystem),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp8] Anton Antonov,
[Text::Plot Raku package](https://github.com/antononcube/Raku-Text-Plot),
(2022),
[GitHub/antononcube](https://github.com/antononcube).

[AAp9] Anton Antonov,
[Text::SubParsers Raku package](https://github.com/antononcube/Raku-Text-SubParsers),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[BDp1] Brian Duggan,
[Jupyter:Kernel Raku package](https://github.com/bduggan/raku-jupyter-kernel),
(2017-2023),
[GitHub/bduggan](https://github.com/bduggan).

[MSp1] Matthew Stuckwisch,
[Intl::Token::Number Raku package](https://github.com/alabamenhu/IntlTokenNumber)
(2021-2023),
[GitHub/alabamenhu](https://github.com/alabamenhu).